In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('..'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
!nvidia-smi

Wed Feb  5 13:33:43 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!pip install transformers

     |████████████████████████████████| 481kB 2.6MB/s 
     |████████████████████████████████| 1.0MB 8.0MB/s 
     |████████████████████████████████| 870kB 19.2MB/s 
     |████████████████████████████████| 3.1MB 19.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=746a0812931352e6ed598174be39b9a7c886c94bb1d1cc210b580db03ee3a88b
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow_hub as hub
import tensorflow as tf
import random
import pickle
import tensorflow.keras.backend as K
import gc
import time
import re
import os
import torch
from scipy.stats import spearmanr
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold,StratifiedKFold
from scipy.stats import spearmanr
from math import floor, ceil
from transformers import AdamW,BertForSequenceClassification


In [0]:
sample_submission = pd.read_csv("/content/drive/My Drive/google-quest-challenge/sample_submission.csv")
test = pd.read_csv("/content/drive/My Drive/google-quest-challenge/test.csv")
train = pd.read_csv("/content/drive/My Drive/google-quest-challenge/train.csv")

MAX_SEQUENCE_LENGTH = 512

In [7]:
print('train shape =', train.shape)
print('test shape =', test.shape)

output_categories = list(train.columns[11:])
input_categories = list(train.columns[[1,2,5]])
print('\noutput categories:\n\t', output_categories)
print('\ninput categories:\n\t', input_categories)

train shape = (6079, 41)
test shape = (476, 11)

output categories:
	 ['question_asker_intent_understanding', 'question_body_critical', 'question_conversational', 'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer', 'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice', 'question_type_compare', 'question_type_consequence', 'question_type_definition', 'question_type_entity', 'question_type_instructions', 'question_type_procedure', 'question_type_reason_explanation', 'question_type_spelling', 'question_well_written', 'answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance', 'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 'answer_type_reason_explanation', 'answer_well_written']

input categories:
	 ['question_title', 'question_body', 'answer']


In [0]:
def _get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def _get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    first_sep = True
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if first_sep:
                first_sep = False 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def _trim_input_q(title, question,max_sequence_length, 
                t_max_len=50, q_max_len=459):

    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    
    t_len = len(t)
    q_len = len(q)

    if (t_len+q_len+3) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            q_max_len = q_max_len + (t_max_len - t_len)
        else:
            t_new_len = t_max_len
      

        q_new_len = q_max_len
            
            
        if t_new_len+ q_new_len+3 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d" 
                             % (max_sequence_length, (t_new_len+a_new_len+q_new_len+4)))
            
        if t_len > t_new_len:
            ind1 = floor(t_new_len/2)
            ind2 = ceil(t_new_len/2)
            t = t[:ind1]+t[-ind2:]
        else:
            t = t[:t_new_len]

        if q_len > q_new_len:
            ind1 = floor(q_new_len/2)
            ind2 = ceil(q_new_len/2)
            q = q[:ind1]+q[-ind2:]
        else:
            q = q[:q_new_len]

    
    return t, q


def _trim_input(title, question, answer, max_sequence_length, 
                t_max_len=30, q_max_len=239, a_max_len=239):

    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    a = tokenizer.tokenize(answer)
    
    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len+q_len+a_len+4) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len)/2)
            q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
        else:
            t_new_len = t_max_len
      
        if a_max_len > a_len:
            a_new_len = a_len 
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
            
            
        if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d" 
                             % (max_sequence_length, (t_new_len+a_new_len+q_new_len+4)))
        

        ## for head + tail
        if t_len > t_new_len:
            ind1 = floor(t_new_len/2)
            ind2 = ceil(t_new_len/2)
            t = t[:ind1]+t[-ind2:]
        else:
            t = t[:t_new_len]

        if q_len > q_new_len:
            ind1 = floor(q_new_len/2)
            ind2 = ceil(q_new_len/2)
            q = q[:ind1]+q[-ind2:]
        else:
            q = q[:q_new_len]

        if a_len > a_new_len:
            ind1 = floor(a_new_len/2)
            ind2 = ceil(a_new_len/2)
            a = a[:ind1]+a[-ind2:]
        else:
            a = a[:a_new_len]
    
    return t, q, a

def _convert_to_bert_inputs(title, question, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken = ["[CLS]"] + title + ["[SEP]"] + question + ["[SEP]"]

    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length)

    return [input_ids, input_masks, input_segments]

def compute_input_arays(df, columns, tokenizer, max_sequence_length):
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in tqdm(df[columns].iterrows()):
        t, q, a = instance.question_title, instance.question_body, instance.answer

        t, q = _trim_input_q(t, q, max_sequence_length)

        ids, masks, segments = _convert_to_bert_inputs(t, q, tokenizer, max_sequence_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
        
    return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]


def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

In [0]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [0]:
import torch

def hard_sigmoid(x):
    """
    Computes element-wise hard sigmoid of x.
    See e.g. https://github.com/Theano/Theano/blob/master/theano/tensor/nnet/sigm.py#L279
    """
    x = (0.2 * x) + 0.5
    x = torch.clamp(x, 0.000001, 0.999999)
    return x

def postProcessing(x):
    x = torch.tensor(x)
    x = torch.cat((torch.sigmoid(x[:,:2]),hard_sigmoid(x[:,2].reshape(-1,1)),torch.sigmoid(x[:,3:11]),hard_sigmoid(x[:,11:13]),\
                        torch.sigmoid(x[:,13].reshape(-1,1)),hard_sigmoid(x[:,14:16]),torch.sigmoid(x[:,16:])),1)
    
    return x.numpy()

In [0]:
class TextDataset(torch.utils.data.TensorDataset):

    def __init__(self, x_train, idxs, targets=None):
        self.input_ids = x_train[0][idxs]
        self.input_masks = x_train[1][idxs]
        self.input_segments = x_train[2][idxs]
        self.targets = targets[idxs] if targets is not None else np.zeros((x_train[0].shape[0], 30))

    def __getitem__(self, idx):
#         x_train = self.x_train[idx]
        input_ids =  self.input_ids[idx]
        input_masks = self.input_masks[idx]
        input_segments = self.input_segments[idx]

        target = self.targets[idx]

        return input_ids, input_masks, input_segments, target

    def __len__(self):
        return len(self.input_ids)

In [0]:
from transformers import BertTokenizer,BertConfig,get_linear_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup

In [13]:
pretrained_weights = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)

In [14]:
x_train = compute_input_arays(train, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)
y_train = compute_output_arrays(train, output_categories)
x_test = compute_input_arays(test, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)

6079it [00:20, 302.92it/s]
476it [00:01, 302.60it/s]


In [0]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss
from transformers import BertPreTrainedModel,BertModel


class CustomizedBert(torch.nn.Module):


    def __init__(self):

        super(CustomizedBert, self).__init__()
        bert_config = BertConfig.from_pretrained(pretrained_weights) 
        bert_config.output_hidden_states=True
        self.bert = BertModel.from_pretrained(pretrained_weights)
        self.dropout = nn.Dropout(bert_config.hidden_dropout_prob)
        self.linear1 = nn.Linear(768*2, 21)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )


        avg_pool = torch.mean(outputs[0], 1)
        max_pool, _ = torch.max(outputs[0], 1)
        pooled_output = self.dropout(torch.cat((max_pool, avg_pool), 1))
        logits = self.linear1(pooled_output)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [0]:
import numpy as np

class callback:
    def __init__(self):
        self.score = list()
        self.model = list()
        self.data = list()
    
    def put(self, model,data, score):
        self.score.append(score)
        self.model.append(model)
        self.data.append(data)

    def get_model(self):
        ind = np.argmin(self.score)
        return self.model[ind]
    def get_data(self):
        ind = np.argmin(self.score)
        return self.data[ind]


In [0]:
with open("/content/drive/My Drive/qa_model/model0202_q2.pkl","rb") as f:
    model_list = pickle.load(f)

In [18]:
len(model_list)

5

In [19]:
NFOLDS = 5
BATCH_SIZE = 4
EPOCHS = 4
SEED = 9638
num_warmup_steps = 100
lr = 3e-5


gradient_accumulation_steps = 1
seed_everything(SEED)


y_oof_q = np.zeros((len(train), 21))
test_pred_q = np.zeros((len(test), 21))

kf = KFold(n_splits=NFOLDS, shuffle=True)

test_loader = torch.utils.data.DataLoader(TextDataset(x_test, test.index),batch_size=BATCH_SIZE, shuffle=False)


for i, (train_idx, valid_idx) in enumerate(kf.split(x_train[0])):
    

    print(f'fold {i+1}')
    gc.collect()
    
    ## loader
    val_loader = torch.utils.data.DataLoader(TextDataset(x_train, valid_idx, y_train),batch_size=BATCH_SIZE, shuffle=False)
    

    net = model_list[i]
    net.cuda()


    valid_preds = np.zeros((len(valid_idx), 21))
        
    net.eval()
    for j,data in enumerate(val_loader):

        # get the inputs
        input_ids, input_masks, input_segments, labels = data
        pred = net(input_ids = input_ids.long().cuda(),
                        labels = None,
                        attention_mask = input_masks.cuda(),
                        token_type_ids = input_segments.long().cuda()
                        )[0]


        valid_preds[j * BATCH_SIZE:(j+1) * BATCH_SIZE] = pred.cpu().detach().numpy()

    y_oof_q[valid_idx] = valid_preds
    
    result = list()


    net.eval()
    with torch.no_grad():
        for data in test_loader:
            input_ids, input_masks, input_segments, labels = data
            y_pred = net(input_ids = input_ids.long().cuda(),
                                labels = None,
                                attention_mask = input_masks.cuda(),
                                token_type_ids = input_segments.long().cuda(),
                            )[0]

            result.extend(y_pred.cpu().detach().numpy())
            
    test_pred_q += np.array(result)/NFOLDS


        


fold 1
fold 2
fold 3
fold 4
fold 5


In [0]:
def _get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def _get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    first_sep = True
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if first_sep:
                first_sep = False 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def _trim_input(title, question, answer, max_sequence_length, 
                t_max_len=30, q_max_len=239, a_max_len=239):

    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    a = tokenizer.tokenize(answer)
    
    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len+q_len+a_len+4) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len)/2)
            q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
        else:
            t_new_len = t_max_len
      
        if a_max_len > a_len:
            a_new_len = a_len 
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
            
            
        if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d" 
                             % (max_sequence_length, (t_new_len+a_new_len+q_new_len+4)))
        
        if t_len > t_new_len:
            ind1 = floor(t_new_len/2)
            ind2 = ceil(t_new_len/2)
            t = t[:ind1]+t[-ind2:]
        else:
            t = t[:t_new_len]

        if q_len > q_new_len:
            ind1 = floor(q_new_len/2)
            ind2 = ceil(q_new_len/2)
            q = q[:ind1]+q[-ind2:]
        else:
            q = q[:q_new_len]

        if a_len > a_new_len:
            ind1 = floor(a_new_len/2)
            ind2 = ceil(a_new_len/2)
            a = a[:ind1]+a[-ind2:]
        else:
            a = a[:a_new_len]
    
    return t, q, a

def _convert_to_bert_inputs(title, question, answer, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken = ["[CLS]"] + title + ["[SEP]"] + question + ["[SEP]"] + answer + ["[SEP]"]

    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length)

    return [input_ids, input_masks, input_segments]

def compute_input_arays(df, columns, tokenizer, max_sequence_length):
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in tqdm(df[columns].iterrows()):
        t, q, a = instance.question_title, instance.question_body, instance.answer

        t, q, a = _trim_input(t, q, a, max_sequence_length)

        ids, masks, segments = _convert_to_bert_inputs(t, q, a, tokenizer, max_sequence_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
        
    return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]


def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

In [21]:
x_train = compute_input_arays(train, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)
x_test = compute_input_arays(test, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)

6079it [00:36, 167.50it/s]
476it [00:03, 156.88it/s]


In [0]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss
from transformers import BertPreTrainedModel,BertModel


class CustomizedBert(torch.nn.Module):


    def __init__(self):

        super(CustomizedBert, self).__init__()
        bert_config = BertConfig.from_pretrained(pretrained_weights) 
        bert_config.output_hidden_states=True
        self.bert = BertModel.from_pretrained(pretrained_weights)
        self.dropout = nn.Dropout(bert_config.hidden_dropout_prob)
        self.linear1 = nn.Linear(768*2, 9)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )


        avg_pool = torch.mean(outputs[0], 1)
        max_pool, _ = torch.max(outputs[0], 1)
        pooled_output = self.dropout(torch.cat((max_pool, avg_pool), 1))
        logits = self.linear1(pooled_output)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [0]:
with open("/content/drive/My Drive/qa_model/model0202_a2.pkl","rb") as f:
    model_list = pickle.load(f)

In [24]:
NFOLDS = 5
BATCH_SIZE = 4
EPOCHS = 4
SEED = 9638 ## the seed of model0202_a2
num_warmup_steps = 100
lr = 3e-5


gradient_accumulation_steps = 1
seed_everything(SEED)


y_oof_a = np.zeros((len(train), 9))
test_pred_a = np.zeros((len(test), 9))

kf = KFold(n_splits=NFOLDS, shuffle=True)

test_loader = torch.utils.data.DataLoader(TextDataset(x_test, test.index),batch_size=BATCH_SIZE, shuffle=False)


for i, (train_idx, valid_idx) in enumerate(kf.split(x_train[0])):
    

    print(f'fold {i+1}')
    gc.collect()
    
    ## loader
    val_loader = torch.utils.data.DataLoader(TextDataset(x_train, valid_idx, y_train),batch_size=BATCH_SIZE, shuffle=False)
    

    net = model_list[i]
    net.cuda()


    valid_preds = np.zeros((len(valid_idx), 9))
        
    net.eval()
    for j,data in enumerate(val_loader):

        # get the inputs
        input_ids, input_masks, input_segments, labels = data
        pred = net(input_ids = input_ids.long().cuda(),
                        labels = None,
                        attention_mask = input_masks.cuda(),
                        token_type_ids = input_segments.long().cuda()
                        )[0]


        valid_preds[j * BATCH_SIZE:(j+1) * BATCH_SIZE] = pred.cpu().detach().numpy()

    y_oof_a[valid_idx] = valid_preds
    
    result = list()


    net.eval()
    with torch.no_grad():
        for data in test_loader:
            input_ids, input_masks, input_segments, labels = data
            y_pred = net(input_ids = input_ids.long().cuda(),
                                labels = None,
                                attention_mask = input_masks.cuda(),
                                token_type_ids = input_segments.long().cuda(),
                            )[0]

            result.extend(y_pred.cpu().detach().numpy())
            
    test_pred_a += np.array(result)/NFOLDS


        


fold 1
fold 2
fold 3
fold 4
fold 5


In [0]:
y_oof = np.concatenate((y_oof_q,y_oof_a), axis=1)

In [29]:
y_oof

array([[ 2.93363047,  1.12548411, -2.5730741 , ..., -2.24046254,
         1.30352974,  3.06612873],
       [ 2.66859078,  1.30669618, -3.52617574, ..., -4.81498289,
        -0.34807563,  2.73056793],
       [ 2.51805305, -0.54163224, -5.48230886, ..., -1.16979015,
        -0.34471586,  3.33662391],
       ...,
       [ 2.04946923,  0.43975094, -5.66392279, ..., -1.76347637,
         0.57397175,  1.44047928],
       [ 2.6926918 ,  0.7509892 ,  0.23309958, ..., -2.04652977,
         2.81085157,  2.69586039],
       [ 3.02421236,  1.47668231, -2.23732352, ..., -3.88071632,
         3.14613461,  2.69052005]])

In [0]:
test_pred = np.concatenate((test_pred_q,test_pred_a), axis=1)

In [0]:
with open("/content/drive/My Drive/qa_model/y_oof_bert.pkl","wb") as f:
    pickle.dump(y_oof,f)

In [0]:
test_pred.shape

In [0]:
y_oof.shape

In [0]:
oof_score = 0
for i in range(30):
    oof_score += np.nan_to_num(
            spearmanr(y_train[:, i], y_oof[:, i]).correlation / 30)   

In [28]:
print("Out of folds score = {}",oof_score)

Out of folds score = {} 0.409722898587619


In [0]:
print("Out of folds score = {}",oof_score)

Out of folds score = {} 0.4097228870494785


In [0]:
def f_0(x):
    if x <= 0.1666:
        x = 0.0000001
    if x >= 0.94:
        x = 0.99999999
    return x

In [0]:
a = torch.sigmoid(torch.tensor(y_oof)).numpy()

In [0]:
a

array([[0.94948409, 0.75500454, 0.07089156, ..., 0.09617533, 0.78642843,
        0.95547376],
       [0.93514762, 0.78695978, 0.02857656, ..., 0.00804216, 0.41384915,
        0.93880647],
       [0.92539776, 0.36780796, 0.00414248, ..., 0.23689292, 0.41466439,
        0.96566408],
       ...,
       [0.88589398, 0.60819968, 0.00345689, ..., 0.14635548, 0.63967913,
        0.80852886],
       [0.93659402, 0.6793942 , 0.55801245, ..., 0.1144035 , 0.94325941,
        0.93678193],
       [0.95365605, 0.81407094, 0.09644853, ..., 0.0202188 , 0.95875614,
        0.93646493]])

In [0]:
a[:,13] = np.array(list(map(f_0,a[:,13])))

In [0]:
spearmanr(y_train[:,13], a[:,13]).correlation

0.14692425170540818

In [0]:
spearmanr(y_train[:, 13], a[:, 13]).correlation

0.14692425170540818

In [0]:
a = postProcessing(a)

In [0]:
oof_score = 0
for i in range(30):
    oof_score += np.nan_to_num(
            spearmanr(y_train[:, i], a[:, i]).correlation / 30)

In [0]:
print("Out of folds score = {}",oof_score)

Out of folds score = {} 0.4348075468555518


In [0]:
print("Out of folds score = {}",oof_score)

Out of folds score = {} 0.41362842711845393


In [0]:
print("Out of folds score = {}",oof_score)

Out of folds score = {} 0.409722898587619


In [0]:
def postProcessing(x):

    def f_0(x):
        if x <= 0.1666:
            x = 0.0000001
        if x >= 0.833:
            x = 0.99999999
        return x

    def f_1(x):
        if x <= 0.1666:
            x = 0.00000001
        # if x >= 0.8333:
        #     x = 0.99999999
        return x

    x[:,2] = np.array(list(map(f_0,x[:,2])))
    x[:,5] = np.array(list(map(f_0,x[:,5])))
    x[:,11] = np.array(list(map(f_0,x[:,11])))
    x[:,15] = np.array(list(map(f_0,x[:,15])))

    x[:,8] = np.array(list(map(f_1,x[:,8])))
    x[:,12] = np.array(list(map(f_1,x[:,12])))
    x[:,14] = np.array(list(map(f_1,x[:,14])))

    return x

    

In [0]:
sample_submission.loc[:, output_categories] = test_pred
sample_submission.to_csv('/content/drive/My Drive/qa_submission/submission0202_2.csv', index=False)